# Orchestration layer

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.agents import initialize_agent, AgentType
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents import AgentExecutor
from langchain.agents import create_react_agent

from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser, JsonKeyOutputFunctionsParser
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.utilities import SerpAPIWrapper
from langchain.tools import Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools import tool
from langchain.tools import Tool

from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List, Dict, Optional

import requests
import datetime
import pytz
import math
# Load environment variables from .env file
load_dotenv()

In [45]:
# Set up OpenAI model (Make sure to set OPENAI_API_KEY in your environment)
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Define a simple calculator tool
def calculate(expression: str) -> str:
    """Evaluates a mathematical expression safely."""
    try:
        return str(eval(expression, {"__builtins__": {}}, {"sqrt": math.sqrt}))
    except Exception as e:
        return f"Error: {str(e)}"

# Define a web search tool (Requires SerpAPI Key)
#search_tool = SerpAPIWrapper()
search_tool2 = TavilySearchResults()

# Create LangChain tools
tools = [
    Tool(name="Calculator", func=calculate, description="Performs mathematical calculations."),
    #Tool(name="Web Search", func=search_tool.run, description="Fetches the latest information from the web."),
    Tool(name="Tavily Search", func=search_tool2.run, description="Searches for the latest information on the web."),
]

# Initialize a ReAct agent
# A higher-level helper function for quickly creating an agent.
# Internally constructs an AgentExecutor with predefined settings.
# Good for quick prototyping but less flexible than manually setting up an AgentExecutor.


agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [ ]:
agent

In [ ]:
# Test the agent
question = ""
response = agent.run(question)

In [ ]:
# Test the agent
question = "What is the temperature in medellin now in celsius?"
response = agent.run(question)

In [ ]:
question = "tell me how much is the price of a flight for next week from medellin to rio de janeiro in COP"
response = agent.run(question)